In [1]:
import requests as rq
import csv
import bs4
import datetime
from urllib.parse import quote
import pandas as pd

In [2]:
google_cache = r"http://webcache.googleusercontent.com/search?q=cache:"
device_list = r"https://phonedb.net/sitemap/"


#get the xml file from the sitemap

sitemap = rq.get(device_list)
soup = bs4.BeautifulSoup(sitemap.text, "lxml")
# get only devices from the sitemap which is 2017-12-30T22:10:18+01:00 and newer using the lastmod tag

locs = []
for url in soup.findAll("url"):
    # parse the lastmod tag to a string and compare it to the date
    date = datetime.datetime.strptime(url.lastmod.text, "%Y-%m-%dT%H:%M:%S%z")
    if date > datetime.datetime(2017, 12, 30, 22, 0, 0, 0, datetime.timezone.utc) and url.loc.text.startswith("https://phonedb.net/index.php?m=device&id="):
            locs.append(url.loc.text)


#remove the first 2 urls as they are not devices
locs.pop(0)
locs.pop(0)

print("There are " + str(len(locs)) + " devices to scrape")


c:\Users\RenzCute\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


There are 9997 devices to scrape


In [9]:
# for devices in locs:
device = locs[66]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'Referer': 'https://www.google.com/'
}

# get the device page
response = rq.get(f"{google_cache}{quote(device + '&d=detailed_specs')}", headers=headers)
doc = bs4.BeautifulSoup(response.text, 'html.parser')

print(device)

https://phonedb.net/index.php?m=device&id=22626&c=samsung_sm-f946bds_galaxy_z_fold5_5g_global_dual_sim_td-lte_1tb__samsung_q5


In [10]:
# get all the td in which the td does not have colspan=2
# get the tr not the td 

# use pandas to get the table ".canvas table"
table = doc.select_one(".canvas table")
df = pd.read_html(str(table))[0]

# convert the table to a dataframe

df

,0,1
0,Introduction:,Introduction:
1,Brand,Samsung
2,Model,SM-F946B/DS Galaxy Z Fold5 5G Global Dual SIM ...
3,Brief Galaxy Z Fold5 lives up to its name by f...,Brief Galaxy Z Fold5 lives up to its name by f...
4,Released,2023 Aug 11
...,...,...
243,Price,3149
244,NaN,Australian Dollar
245,Datasheet Attributes:,Datasheet Attributes:
246,Data Integrity,Final
